gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

!pip install cellpose

from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [1]:
# Import modules
import numpy as np
import time, os, sys, random
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
from cellpose import models
from cellpose.io import imread
from cellpose import plot
from skimage.util import img_as_ubyte
import pandas as pd
from skimage.measure import label, regionprops
from skimage.measure import label, regionprops
from skimage import measure

ImportError: Numba needs NumPy 1.20 or less

In [ ]:
import scipy
from scipy import ndimage as ndi
import cv2
from skimage.morphology import square, dilation
from sklearn.cluster import KMeans, DBSCAN
# https://stackoverflow.com/questions/19970764/making-feature-vector-from-gabor-filters-for-classification
# https://stackoverflow.com/questions/20608458/gabor-feature-extraction

# Image segmentation

In [ ]:
#------------------------------------------------------
# Number of images we want to read. Only in test phase. If we want to read all images we will use len(Cherry_file) in for loop below
nb_images = 2

# Get path to folders containing images
Image_Directory = os.getcwd() + "\\20151218_E14_BM_mTF8additionals_4"
#Image_Directory = rC:\Users\Claudia\Desktop\Master EPFL\machine learning\project_2  #Easier for me (Alexis) than finding of to get my path file nicely
#Image_Directory = "/content/gdrive/MyDrive/ML_2_transcripted_cherry/TF1"
# other folders: 
# \\20151218_E14_BM_mTF8additionals_4

print(Image_Directory)

# Calculate number of document-> array containing 

# Create Lists containing datapath to all images of Cherry/ Ypet files
image_format = "tif"
cherry_condition = "TexasRed"
ypet_condition = "YFP"
Cherry_file = []
Ypet_file = []
test = []
TF_name_tot = []

# Walk through directory (call all files) and assign them to proper list
for root, dirs, files in os.walk(Image_Directory, topdown=False): # root = given in Input, dirs = folders, files= files
    for f in files:
        if f.endswith(image_format) and cherry_condition in f: 
            test.append(f)
            TF_name_tot.append(f.partition("(")[0])
            Cherry_file.append(os.path.join(root, f))
        elif f.endswith(image_format) and ypet_condition in f: 
            Ypet_file.append(os.path.join(root, f))

#assert len(Cherry_file) != len(Ypet_file), f"There are {len(Cherry_file)} Cherry files and {len(Ypet_file)} Ypet files. The number has to be equal, check manually what's missing"


# Initialize lists containing cherry, ypet images. Length = nb_images or len(Cherry_file) if we want to read all images
cherry_imgs = np.ndarray(nb_images,dtype = np.ndarray)
ypet_imgs = np.ndarray(nb_images,dtype = np.ndarray)


# Read image (We donnot add it in for loop above to enable reading subset of image)
for i in range(nb_images): #len(Cherry_file):
    cherry_imgs[i] = imread(Cherry_file[i])
    ypet_imgs[i] = imread(Ypet_file[i])
    
cherry_imgs = cherry_imgs.tolist()
ypet_imgs = ypet_imgs.tolist()

In [ ]:
plt.imshow(imread(Ypet_file[0]))

In [ ]:
# Definine parameters for cellpose module using nuclei modus
# https://cellpose.readthedocs.io/en/latest/settings.html
channels = [0,0] # Corresponds to greyscale image
diameter = None # Model was trained on diameters of 17 pixels. None-> automated estimation of the diameter. Can be changed by hand later
flow_threshold = 0.4 # maximum allowed error of the flows for each mask, default = 0.4
gpu = False
model_type='nuclei'

# Set model
model = models.Cellpose(gpu, model_type)

# Nuclei segmentation of all images within cherry_imgs list
masks, flows, styles, diams = model.eval(cherry_imgs, diameter=diameter, channels=channels, flow_threshold=flow_threshold, do_3D=False)


In [ ]:
'''
from cellpose import io
import os
#os.mkdir("/content/gdrive/MyDrive/ML_2_transcripted_cherry/masks_TF1/A_1")
files_names = ["/content/gdrive/MyDrive/ML_2_transcripted_cherry/masks_TF1/A_1/img_"+str(i+1) for i in range(nb_images)]
io.masks_flows_to_seg(cherry_imgs, masks, flows, diams, files_names, channels)'''

In [ ]:
# Display results
image_i = 0 # Number of image we want to display. Max value= nb_images or len(Cherry_file)
fig = plt.figure(figsize=(24,8))
plot.show_segmentation(fig, cherry_imgs[image_i], masks[image_i], flows[image_i][0], channels)
plt.tight_layout()
plt.show()

***
# Feature extraction

In [ ]:
def gabor_feature_extraction(kernels,patch_data):
    # Apply Gabor filter: We receive per kernel a response. We stack all the response to one matrix---------------------------------------------------------
    image = patch_data
    g_response = []

    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        g_response.append(filtered)
    n_clusters = len(g_response)
    g_response = np.vstack(g_response)

    # Cluster Gabor response with k-means. Find which part of the pixel belongs to which Gabor response ---------------------------------------------------------
    random_state = 1
    random.seed(random_state)
    kmean = KMeans(n_clusters, random_state=random_state).fit(g_response) #

    # Texton map: Assign each pixel to the identity of the closest cluster center---------------------------------------------------------
    histogram = np.histogram(kmean.labels_, bins = len(np.unique(kmean.labels_)))
    # Normalize
    gabor_feature = histogram[0] / histogram[0].sum()
    return gabor_feature

In [ ]:
def feature_binary_pattern(patch_data):
    # Apply lbp
    METHOD = 'uniform' #  'default'
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(patch_data, n_points, radius, METHOD)
    
    # Create histogram
    count = len(np.unique(lbp))
    H, bins = np.histogram(lbp, count)

    # Drop background values
    w = width = radius - 1
    index_background = n_points - w
    H = H[0: index_background]
    bins =  bins[0: index_background]

    # plt.bar(bins,H,width=1)
    
    # Normalize
    lbp_feature = H / H.sum()
    return lbp_feature

In [ ]:
from skimage.feature import local_binary_pattern
from skimage.color import label2rgb
from skimage.transform import rotate

In [ ]:
# Initialize list of features 
columnname = ["" for i in range(1)]
patch_size = [0 for i in range(1)]
mean_intensity = [0 for i in range(1)]
sum_intensity = [0 for i in range(1)]
median_intensity = [0 for i in range(1)]
variance_intensity = [0 for i in range(1)]
imgs = [-1 for i in range(1)]

In [ ]:
# prepare Gabor filter bank kernels---------------------------------------------------------
kernels = []
sigmas = [1,3]
tilts = [0,0.25,0.5,0.75] # perpendicular, diagonal, horizontal, other diagonal 
Wavelengths = [0.25,0.75] 
gamma =  0.5 # Spatial aspect ratio
psi = 0 # Phase offset.

for tilt in tilts:
    for sigma in sigmas:
        for lambd in Wavelengths:
            kernel = cv2.getGaborKernel((10,10), sigma, tilt, lambd, gamma, psi, ktype = cv2.CV_32F)
            kernels.append(kernel)

In [ ]:
from skimage.feature import local_binary_pattern

In [ ]:
# Extract feature of each nucleus within images nb_images
list_of_all_images = []

for j in range(nb_images):
    
    # Initialize features: Faster then appending values to list with append
    mask =  masks[j]  
    TF_name = TF_name_tot[j]
    start_nucleus = 10 #1 # we have to at least start from 1, otherwise we take whole mask
    end_nucleus =  20 #len(np.unique(mask)) # len(np.unique(mask))  #IMAGE SPECIFIC: cases where cell are big enough (sell last cell of the notebook)
    range_s = range(start_nucleus, end_nucleus)
    imgs = [j for i in range_s]
    ypet = ypet_imgs[j]
    columnname = ["" for i in range_s]
    blur = [False for i in range_s]
    patch_size = [0 for i in range_s]
    mean_intensity = [0 for i in range_s]
    sum_intensity = [0 for i in range_s]
    median_intensity = [0 for i in range_s]
    std_intensity = [0 for i in range_s]
    var_intensity = [0 for i in range_s]
    skew_intesity = [0 for i in range_s]
    kurt_intesity = [0 for i in range_s]
    iqr_intesity = [0 for i in range_s]
    entropy_intesity = [0 for i in range_s]
    gabor_features = [0 for i in range_s]
    lbp_features = [0 for i in range_s]
    blur_lapl = [False for i in range_s]
    blur_ski = [False for i in range_s]
                  
    #calculate different properties of the mask
    props = regionprops(mask)
    
    #initialize arrays to store information from the mask
    perimeter = [0 for i in range_s]
    feret_diameter_max = [0 for i in range_s]
    area_test = [0 for i in range_s]
    axis_major = [0 for i in range_s]
    axis_minor = [0 for i in range_s]
    solidity = [0 for i in range_s]

    '''
    # Iterate to explore and find good threshold in exploration phase
    patch_data_list = []
    variance_data_list = []'''
    # Threshold for blur detection
    var_threshold = 10000000
    blur_threshold = 0.7
    
    # Iterate through all nucleus: Take submatrix, Calculate patch size and corresponding ypet signal (mean) and store it into ypet_intensity
    iter_i = 0 # Ensures that no 0 are saved where patch is too small/big
    
    for i in range_s:# # Iterate through all nucleus: in range(start_nucleus, end_nucleus)
        cur = ypet[mask==i]
        patch_size_ = np.count_nonzero(~np.isnan(cur))
        
        # Extract subimage ----------------------------------------------------------------
        # Extract subimage of nucleus (Not just its values, which is done above)
        boolean = np.where(mask==i, 1,False) # Boolean matrix where patch = 1, else 0
        patch_values = ypet*boolean
        # Extract submatrix with a buffer of 3 pixels to calculate the blurrness
        variance_boolean = dilation(boolean, square(7))
        variance_values = ypet * variance_boolean
        
        # Drop pixels without nucleus information ----------------------------------------------------------------
        # Initialize conditions on when a column/ row should be dropped
        drop_nucleus = False
        drop_column = []
        drop_column_variance = []
        drop_row = []
        drop_row_variance = []
        
        
        # Iterate through all columns to find nucleus that is on the border and the columns that only contain mask
        for column in range(boolean.shape[1]):
            
            # drop nucleus if it's on the border 
            if (((column == 0) | (column == boolean.shape[1]-1)) & (np.any(boolean[:,column])== True)):
                drop_nucleus = True
                break 
                
            # drop if column has no information about nucleus: first check for broader variance_submatrix (Matrix with a buffer around submatrix to calculate variance). Then smaller submatrix 
            if np.any(variance_boolean[:,column])== False:
                drop_column_variance.append(column) 
                drop_column.append(column) 
            elif np.any(boolean[:,column])== False: 
                drop_column.append(column) 
        
        # Iterate through all columns to find nucleus that is on the border and the columns that only contain mask
        for row in range(boolean.shape[0]):
            
            # drop nucleus if it's on the border 
            if (((row == 0) | (row == boolean.shape[0]-1)) & (np.any(boolean[row,:])== True)):
                drop_nucleus = True
                break
                
            # drop if row has no information about nucleus:  first check for broader variance_submatrix  
            if np.any(variance_boolean[row,:])== False:
                drop_row.append(row)
                drop_row_variance.append(row)
            elif np.any(boolean[row,:])== False: 
                drop_row.append(row)
        
        # Drop pixels without nucleus information
        patch_data = np.delete(patch_values,drop_column,1)  
        patch_data = np.delete(patch_data,drop_row,0) 
        variance_data = np.delete(ypet,drop_column_variance,1)  
        variance_data = np.delete(variance_data,drop_row_variance,0) 
        
        '''
        # Exploration phase
        patch_data_list.append(patch_data)
        variance_data_list.append(variance_data)'''
        
        # Calculate blurness based on laplacian----------------------------------------------------------------
        var_lapl = cv2.Laplacian(variance_data, cv2.CV_64F).var()
        if var_lapl >= var_threshold: # nucleus is blurry
            blured_nucleus = True
        else: 
            blured_nucleus = False
        
        # Calculate blurness based on skimage----------------------------------------------------------------
        blur_ski_nucleus = measure.blur_effect(variance_data)
        if blur_ski_nucleus >= blur_threshold:
            blur_ski_temp = True
        else:
            blur_ski_temp = False
        
        # Feature calculation ----------------------------------------------------------------
        
        if ((patch_size_ > 10) & (patch_size_ < 50000) & (drop_nucleus== False)): 
            # extract intensity feature
            columnname[iter_i] = "patch_" + str(iter_i)    #We should not forgot that there is a -1 here. (when it is here)               
            patch_size[iter_i] = patch_size_
            blur_lapl[iter_i] = blured_nucleus
            blur_ski[iter_i] = blur_ski_temp
            sum_intensity[iter_i] = np.sum(np.sum(cur))
            mean_intensity[iter_i] = np.nanmean(cur)
            median_intensity[iter_i] = np.nanmedian(cur)
            
            std_intensity[iter_i] = np.nanstd(cur)
            var_intensity[iter_i] = np.nanvar(cur)
            skew_intesity[iter_i] = scipy.stats.skew(cur)
            kurt_intesity[iter_i] = scipy.stats.kurtosis(cur)
            iqr_intesity[iter_i] = scipy.stats.iqr(cur)
            entropy_intesity[iter_i] = scipy.stats.entropy(cur)
            
            gabor_features[iter_i] = gabor_feature_extraction(kernels,patch_data) # List of two arrays, first nb of counts in histogram, second: bin number
            temp_lbp = feature_binary_pattern(patch_data)
            lbp_features[iter_i]  = temp_lbp
            lbp_count = len(temp_lbp)
            
            perimeter[iter_i] = props[i-1].perimeter
            feret_diameter_max[iter_i] = props[i-1].feret_diameter_max
            area_test[iter_i] = props[i-1].area
            axis_major[iter_i] = props[i-1].major_axis_length
            axis_minor[iter_i] = props[i-1].minor_axis_length
            solidity[iter_i] = props[i-1].solidity
            
            iter_i +=1
    
    # Drop empty rows: some patches values were not calculated because their size is too small/big 
    if iter_i != range_s: # Some patches were not considered -> we have to drop those rows
        mean_intensity = mean_intensity[:iter_i]
        columnname = columnname[:iter_i]
        blur_lapl = blur_lapl[:iter_i]
        blur_ski = blur_ski[:iter_i]
        patch_size = patch_size[:iter_i]
        sum_intensity = sum_intensity[:iter_i]
        median_intensity = median_intensity[:iter_i]
        imgs = imgs[:iter_i]
        std_intensity = std_intensity[:iter_i] 
        var_intensity = var_intensity[:iter_i] 
        skew_intesity = skew_intesity[:iter_i] 
        kurt_intesity = kurt_intesity[:iter_i] 
        iqr_intesity = iqr_intesity[:iter_i]
        entropy_intesity = entropy_intesity[:iter_i] 
        gabor_features = gabor_features[:iter_i] 
        lbp_features = lbp_features[:iter_i] 
        
        perimeter = perimeter[:iter_i]
        feret_diameter_max = feret_diameter_max[:iter_i]
        area_test = area_test[:iter_i]
        axis_major = axis_major[:iter_i]
        axis_minor = axis_minor[:iter_i]
        solidity = solidity[:iter_i]
        
    ypet_intensity = pd.DataFrame(index=columnname)
    ypet_intensity['img'] = imgs
    ypet_intensity['TF_name'] = TF_name
    ypet_intensity['blur_lapl'] = blur_lapl
    ypet_intensity['blur_ski'] = blur_ski
    ypet_intensity['patch_size'] = patch_size
    ypet_intensity['mean_intensity'] = mean_intensity
    ypet_intensity['sum_intensity'] = sum_intensity
    ypet_intensity['median_intensity'] = median_intensity
    
    ypet_intensity['standard_deviation'] = std_intensity 
    ypet_intensity['variance'] = var_intensity
    ypet_intensity['skewness'] = skew_intesity
    ypet_intensity['kurtosis'] = kurt_intesity
    ypet_intensity['interquartile_range ']= iqr_intesity
    ypet_intensity['entropy'] = entropy_intesity
    ypet_intensity['Perimeter'] = perimeter 
    ypet_intensity['area_Test'] = area_test
    ypet_intensity['axis_major_length'] = axis_major
    ypet_intensity['feret_diameter_max'] = feret_diameter_max
    ypet_intensity['axis_minor_length'] = axis_minor
    ypet_intensity['solidity'] = solidity
    
    list_of_all_images.append(ypet_intensity)
    
    if imgs == []:
        print('All nucleis have been droppped. In order to extract features you need nucleus that are not by the border and not blurry.')
    else: 
        gabor_features_tot = np.vstack(gabor_features)
        lbp_features_tot = np.vstack(lbp_features)

        for filter_bank in range (len(kernels)):
            name = 'gabro_'+ str(filter_bank)
            ypet_intensity[name] = gabor_features_tot[:,filter_bank]


        for lbp_features_count in range (lbp_features_tot.shape[1]):
            name = 'lbp_' + str(lbp_features_count)
            ypet_intensity[name] = lbp_features_tot[:,lbp_features_count]
                                      
   
    #ypet_intensity_glob = pd.concat([ypet_intensity_glob,ypet_intensity])
    print("step "+str(j)+" over "+str(nb_images)+ " is done!")
    
whole array = pd.concat(list_of_all_images)    
    

### Add column with circularity

Formula for circularity: 4*pi*area/(perimeter^2)
Zero would mean that it is a circle. Everything else is between zero and one.

In [ ]:
ypet_intensity['Circularity'] = (4*np.pi*ypet_intensity['patch_size'])/(ypet_intensity['Perimeter']**2)

In [ ]:
ypet_intensity.head()

In [ ]:
'''
path = '/content/drive/MyDrive/Project_2_ml/20151218_E14_BM_mTF8additionals_4/Features/features_cells_TF8_A1.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
    ypet_intensity_glob.to_csv(f)

"""  Was just to create the ~800 folders easier :)
import os
alphabet = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
for k in range(8):
  for j in range(8):
    for i in range(12):
      os.mkdir("/content/gdrive/MyDrive/ML_2_transcripted_cherry/masks_TF"+str(k+1)+"/"+alphabet[j]+"_"+str(i+1))
"""

 to remove the "copie" part of it once I did copy
import os
os.rename("/content/gdrive/MyDrive/ML_2_transcripted_cherry/masks_TF1/A_1/Copie de img_1_seg.npy","/content/gdrive/MyDrive/ML_2_transcripted_cherry/masks_TF1/A_1/img_1_seg.npy")
'''


***
# Preprocessing

### Outlier removal

To adapt is the columns where we want to change the outliers

In [ ]:
columns_outliers = ['mean_intensity','patch_size']
remove_outliers = ypet_intensity.copy()

def outlier_removal(without_outliers, outlier_range = 1.5):
    columns_outliers= ['mean_intensity','patch_size']    
    # Use a for loop to iterate over the columns of the dataframe
    for col in columns_outliers:
        # Look up the threshold value for the current column using the dictionary
        median = without_outliers[col].median()
        q_75 = without_outliers[col].quantile(q = 0.75)
        q_25 = without_outliers[col].quantile(q = 0.25)
        interquantile = q_75 - q_25                              
        upper_bound = median + (interquantile * outlier_range)
        lower_bound = median - (interquantile * outlier_range)

      # Create a boolean mask that is True for rows with a value less than or equal to the threshold
        mask = (without_outliers[col] <= upper_bound) & (without_outliers[col] >= lower_bound)
      # Use the mask to filter the dataframe and keep only the rows with a value less than or equal to the threshold
        without_outliers = without_outliers[mask]
        
    return without_outliers


In [ ]:
without_outliers = remove_outliers.groupby('TF_name',as_index=False).apply(outlier_removal).reset_index()
without_outliers = without_outliers.drop(['level_0'],axis=1)
without_outliers = without_outliers.set_index('level_1')
without_outliers.index.name = None

In [ ]:
ypet_intensity_processed = without_outliers.copy()

### Normalize ypet_intensity for each TF after we've concenated all df

In [ ]:
# Normalize 
rows_to_be_normalized = ['mean_intensity', 'median_intensity']
TF_grouped = ypet_intensity_processed.groupby('TF_name')
Normalized_rows = TF_grouped[rows_to_be_normalized].transform(lambda x: (x - x.mean()) / x.std())

# Replace normalized
ypet_intensity_processed[rows_to_be_normalized] = Normalized_rows

In [ ]:
ypet_intensity_processed

### pca of ypet_intensity, 

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
n_components= 2
pca = PCA(n_components)

# define which columns should be excluded
pca_excluded = ['img', 'TF_name']
to_test = ypet_intensity.drop(pca_excluded, axis = 1)

# Check how much variance the PCA components explain
pca_result = pca.fit(to_test)
print(pca_result.explained_variance_ratio_)



In [ ]:
# apply the dimensionality reduction on data
pca_components = pca.fit_transform(to_test)

# Combine excluded data and pca
#ypet_intensity[pca_excluded].merge(ypet_intensity_pca)
ypet_intensity_pca = ypet_intensity[pca_excluded]
for iter_i in range(n_components):
    name = 'pca_component_' + str(iter_i)
    ypet_intensity_pca[name] = pca_components[:,iter_i]

In [ ]:
ypet_intensity_pca

***

# Task 1

In [ ]:
#cross validation with polynomial expansion
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
import statsmodels.formula.api as smf
import seaborn as sns
import re
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
import statsmodels.api as sm

In [ ]:
# Split train test
df_train, df_test = train_test_split(ypet_intensity, train_size = 0.8, test_size = 0.2, random_state = 10)

testfeature = ['mean_intensity', 'sum_intensity', 'median_intensity']

df_train_x = df_train[[testfeature[0]]]
df_test_x = df_test[[testfeature[0]]]

df_train_y = df_train[['patch_size']]
df_test_y = df_test[['patch_size']]

In [ ]:
#model to select
maxdegree = 5 #To define is the maximum degree of the polynomial
cross_validation_ridge_error = np.zeros(maxdegree-1)
cross_validation_lm_error = np.zeros(maxdegree-1)

#see which degree fits data the best for linear regression
for d in range(1, maxdegree):
    #polynomial feature expansion of x_train
    x_poly_train = PolynomialFeatures(degree=d).fit_transform(df_train_x)
    #apply linear regression model and cross-validation for alpha-ridge regression
    lr = LinearRegression()
    rr = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
    #apply cross validation
    cve = cross_validate(lr,x_poly_train,df_train_y,scoring='neg_mean_squared_error', cv=5, return_train_score=True)
    crr = cross_validate(rr,x_poly_train,df_train_y,scoring='neg_mean_squared_error', cv=5, return_train_score=True)
    #make array for cross validation with linear model and for ridge regression
    cross_validation_lm_error[d-1] = np.mean(np.absolute(cve['test_score']))
    cross_validation_ridge_error[d-1] = np.mean(np.absolute(crr['test_score']))

In [ ]:
index_min_lm = np.argmin(cross_validation_lm_error)
index_min_ridge = np.argmin(cross_validation_ridge_error)
x_poly_test = PolynomialFeatures(degree=index_min_lm+1).fit_transform(df_test_x)
x_poly_train = PolynomialFeatures(degree=index_min_lm+1).fit_transform(df_train_x)

x_poly_train_df = pd.DataFrame(x_poly_train)
x_poly_test_df = pd.DataFrame(x_poly_test)

#make linear model
model_lm = LinearRegression().fit(x_poly_train_df, df_train_y)
model_ridge = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(x_poly_train_df, df_train_y)

#train error
y_train_pred_lm = model_lm.predict(x_poly_train)
mse_train_lm = mean_squared_error(df_train_y,y_train_pred_lm)
y_train_pred_ridge = model_ridge.predict(x_poly_train)
mse_train_ridge = mean_squared_error(df_train_y,y_train_pred_ridge)

#test error
y_test_pred_lm = model_lm.predict(x_poly_test_df)
mse_test_lm = mean_squared_error(df_test_y,y_test_pred_lm)
y_test_pred_ridge = model_ridge.predict(x_poly_test_df)
mse_test_ridge = mean_squared_error(df_test_y,y_test_pred_ridge)

In [ ]:
sns.regplot(x="patch_size", y="mean_intensity", data=ypet_intensity, fit_reg=False)

In [ ]:
#show predicted things
plt.plot(df_train_y,df_train_x, 'ro')
plt.plot(y_test_pred_lm,df_test_x, 'ro',color = 'black')
#plt.yscale("log")
#plt.xscale("log")
plt.show()

In [ ]:
#show predicted things
plt.plot(df_train_y,df_train_x, 'ro')
plt.plot(y_test_pred_ridge,df_test_x, 'ro',color = 'black')
#plt.yscale("log")
plt.xscale("log")
plt.show()

In [ ]:
if mse_test_ridge >= mse_test_lm:
    print('take linear regression')
    print('coefficients: ',model_lm.coef_)
    print('intercept: ',model_lm.intercept_)
    
else:
    print('take ridge regression')
    print('coefficients: ',model_ridge.coef_)
    print('intercept: ',model_ridge.intercept_)
    print('alpha: ',model_ridge.alpha_)

***

## Task 2: Unsupervised clustering

Use pca or standard df

In [ ]:
clustering_feature = ['patch_size', 'sum_intensity']
ypet_intensity[clustering_feature]

In [ ]:
ypet_intensity_pca

In [ ]:
ypet_cluster = ypet_intensity_pca.reset_index().set_index(['index','img', 'TF_name'])
ypet_cluster

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
random_state = 42
n_clusters = 5 #10 # Number of descriptions we want for TF
kmean = KMeans(n_clusters=n_clusters, random_state=random_state).fit(ypet_cluster)

ypet_clustered = ypet_cluster.copy()
ypet_clustered['cluster'] = kmean.labels_

In [ ]:
plt.scatter(ypet_cluster.iloc[:,0], ypet_cluster.iloc[:,1], c=kmean.labels_, alpha=0.6)

In [ ]:
# Find more information
min_n = ypet_clustered.groupby('cluster').count().min()[0] #min number of points i n a cluster
max_n = ypet_clustered.groupby('cluster').count().max()[0] #min number of points i n a cluster
min_n, max_n

In [ ]:
# Extract x percentage of data per cluster -> easier to analyze

extracted_i = []
x = 0.9

for i in range(n_clusters):

    # indices of all the points from X that belong to cluster i
    C_i = np.where(kmean.labels_ == i)[0].tolist() 
    n_i = len(C_i) # number of points in cluster i

    # indices of the points from X to be sampled from cluster i
    sample_i = np.random.choice(C_i, math.ceil((x * n_i))).tolist()
    extracted_i.extend(sample_i) 
    

In [ ]:
ypet_clustered.iloc[np.unique(extracted_i)]

***
# Visualizations (Exploration phase)

#### Tune blur

plt.imshow(cv2.Laplacian(variance_data_list[test], cv2.CV_64F), cmap='gray')


for i in range(1,100), image E-7

Not blurred:
variance_data_list[-1] -> 1'763'695
variance_data_list[-10] -> 3'056'590
variance_data_list[-10] -> 3'056'590
[-17] -> 1'056'899
[-20] -> 567'588
[-21] -> 2'138'449
[-23] -> 5'134'705
[-33] 30'821'019


Right classified with other nucleus near:
variance_data_list[-6] -> 25'095'309
variance_data_list[-13] ->5'698'016
[-16] -> 9'219'300
[-19] -> 11'049'582
[-25] -> 2'313'727
[-26] -> 157'114'853
[-27] -> 10'010'503
[-28] -> 3'063'935
[-31] -> 1'803'048
9'826'707


Wrongly classified:
variance_data_list[-6] -> 25'095'309
variance_data_list[-9] -> 57'898'307
[-14] -> 21'739'451
[-15] -> 6'769'331
[-30] -> 12'627'256


Overlay:
variance_data_list[-4] -> 34'614'946
[-22] -> 12'964'725
[-29] -> 17'727'949

Blurr/ overflow:
[-18] -> 283'062'008
[-24] -> 17'935'490

In [ ]:
cv2.Laplacian(variance_data_list[test], cv2.CV_64F).var()

In [ ]:
measure.blur_effect(variance_data_list[test])

#### Mask, nucleus

In [ ]:
plt.imshow(boolean, cmap='gray')

In [ ]:
plt.imshow(patch_data, cmap='gray')

In [ ]:
test = 31
plt.imshow(variance_data_list[test], cmap='gray')

In [ ]:
plt.imshow(patch_data_list[test], cmap='gray')

## Local binary patterns

In [ ]:
# Visualize lpb
METHOD = 'uniform'   
radius = 1
n_points = 8 * radius
image = patch_data

lbp = local_binary_pattern(image, n_points, radius, METHOD)

    
def overlay_labels(image, lbp, labels):
    mask = np.logical_or.reduce([lbp == each for each in labels])
    return label2rgb(mask, image=image, bg_label=0, alpha=0.5)


def highlight_bars(bars, indexes):
    for i in indexes:
        bars[i].set_facecolor('r')
def hist(ax, lbp):
    n_bins = int(lbp.max() + 1)
    return ax.hist(lbp.ravel(), density=True, bins=n_bins, range=(0, n_bins),
                   facecolor='0.5')


# plot histograms of LBP of textures
fig, (ax_img, ax_hist) = plt.subplots(nrows=2, ncols=3, figsize=(9, 6))
plt.gray()

titles = ('edge', 'flat', 'corner')
w = width = radius - 1
edge_labels = range(n_points // 2 - w, n_points // 2 + w + 1)
flat_labels = list(range(0, w + 1)) + list(range(n_points - w, n_points + 2))
i_14 = n_points // 4            # 1/4th of the histogram
i_34 = 3 * (n_points // 4)      # 3/4th of the histogram
corner_labels = (list(range(i_14 - w, i_14 + w + 1)) +
                 list(range(i_34 - w, i_34 + w + 1)))

label_sets = (edge_labels, flat_labels, corner_labels)

for ax, labels in zip(ax_img, label_sets):
    ax.imshow(overlay_labels(image, lbp, labels))

for ax, labels, name in zip(ax_hist, label_sets, titles):
    counts, _, bars = hist(ax, lbp)
    highlight_bars(bars, labels)
    ax.set_ylim(top=np.max(counts[:-1]))
    ax.set_xlim(right=n_points + 2)
    ax.set_title(name)

ax_hist[0].set_ylabel('Percentage')
for ax in ax_img:
    ax.axis('off')



In [ ]:
# Remove background    

# plot histograms of LBP of textures
fig, (ax_img, ax_hist) = plt.subplots(nrows=2, ncols=3, figsize=(9, 6))
plt.gray()

titles = ('edge', 'flat', 'corner')
w = width = radius - 1
edge_labels = range(n_points // 2 - w, n_points // 2 + w + 1)
flat_labels = list(range(0, w + 1)) # Remove background: list(range(n_points - w, n_points + 2))
i_14 = n_points // 4            # 1/4th of the histogram
i_34 = 3 * (n_points // 4)      # 3/4th of the histogram
corner_labels = (list(range(i_14 - w, i_14 + w + 1)) +
                 list(range(i_34 - w, i_34 + w + 1)))

label_sets = (edge_labels, flat_labels, corner_labels)



for ax, labels in zip(ax_img, label_sets):
    ax.imshow(overlay_labels(image, lbp, labels))

for ax, labels, name in zip(ax_hist, label_sets, titles):
    counts, _, bars = hist(ax, lbp)   
    highlight_bars(bars, labels)
    ax.set_ylim(top=np.max(counts[:-1]))
    ax.set_xlim(right=n_points + 2)
    ax.set_title(name)

ax_hist[0].set_ylabel('Percentage')
for ax in ax_img:
    ax.axis('off')

# Drop background
count = len(np.unique(lbp))
H, bins = np.histogram(lbp, count)

# Drop background values
fig, ax = plt.subplots(figsize=(12, 8))
index_background = n_points - w
H = H[0: index_background]
bins =  bins[0: index_background]

plt.bar(bins,H,width=1)